In [9]:
import numpy as np
import pandas as pd
import spotipy as sp
import numbers
import spotipy.util as util

SPOTIPY_REDIRECT_URI='http://localhost:8888/callback'
SPOTIPY_CLIENT_SECRET='c02f97ddd119470d8e24b290722c7490'
SPOTIPY_CLIENT_ID='647ffb8c163b4882be0cafe4bf1352d5'
USERNAME='lh7smhcywr18vctg4xmnoehzu'
SCOPE='user-library-read'

token = util.prompt_for_user_token(username=USERNAME,
                           scope=SCOPE,
                           client_id=SPOTIPY_CLIENT_ID,
                           client_secret=SPOTIPY_CLIENT_SECRET,
                           redirect_uri=SPOTIPY_REDIRECT_URI)

s = sp.Spotify(auth=token)

In [12]:
def get_track_ids(tracks, ide=True):
    """Collect all track id's in a list of tracks"""
    
    if ide:
        return [track['track']['id'] for track in tracks]
    else:
        return [track['id'] for track in tracks]


def get_audio_features(ids):
    """Collect all audio features in a list from a given list of track id's"""
    audio_feats = []

    for i in range(0, len(ids), 100):
        audio_feats += s.audio_features(ids[i:i + 100])

    return audio_feats

def getDataframe(features):
    matrix = []
    metamatrix = []
    for track in features:
        try:
            matrix += [[x for x in track.values() if isinstance(x, numbers.Number)]]
            metamatrix += [[x for x in track.values() if not isinstance(x, numbers.Number)]]
        except:
            pass
        
    return np.array(matrix), metamatrix



def get_random_sample(genre, iters = 20):
    """Main function"""
    query = s.search(q='genre:{0}'.format(genre))
    tracks = []
    
    i = 0
    
    while query['tracks']['next']:
        tracks.extend(query['tracks']['items'])
        query = s.next(query['tracks'])
        
        i += 1
        
        if i == iters:
            break
            
    feature_names = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
     'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
  
    # These steps are not computationally expensive
    track_ids = get_track_ids(tracks, ide=False)
    audio_feats = get_audio_features(track_ids)
    
    matrix_of_features, metamatrix = getDataframe(audio_feats)
    df = pd.DataFrame(matrix_of_features)
    df.columns = feature_names
    
    return df, metamatrix

In [20]:
# Example
a, b = get_random_sample("vaporwave", 1)
a

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.153,0.223,6.0,-24.664,1.0,0.0605,0.929000,0.7700,0.2160,0.1760,77.243,285173.0,4.0
1,0.567,0.743,8.0,-8.614,1.0,0.0626,0.035600,0.8470,0.2210,0.4230,170.083,212881.0,5.0
2,0.356,0.469,6.0,-8.403,0.0,0.1200,0.361000,0.8820,0.3960,0.2310,85.228,302211.0,4.0
3,0.674,0.655,10.0,-6.574,1.0,0.0413,0.473000,0.8440,0.0965,0.0597,174.014,233793.0,4.0
4,0.697,0.662,1.0,-9.824,0.0,0.1730,0.058800,0.6880,0.3150,0.3780,86.991,181149.0,4.0
5,0.192,0.289,5.0,-17.759,1.0,0.0355,0.907000,0.9790,0.1040,0.0385,98.183,260599.0,4.0
6,0.411,0.925,7.0,-1.701,1.0,0.0774,0.000006,0.9500,0.1970,0.2140,131.075,214351.0,4.0
7,0.306,0.794,5.0,-1.657,0.0,0.0634,0.000059,0.0801,0.3000,0.3020,147.163,208973.0,4.0
8,0.645,0.114,9.0,-15.768,1.0,0.3560,0.967000,0.8310,0.1120,0.1910,78.015,86571.0,4.0
9,0.660,0.164,0.0,-18.224,0.0,0.1130,0.743000,0.8500,0.1250,0.4300,77.004,115299.0,4.0
